## Langchain: function calling and structured output

In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# Load environment variables from .env
load_dotenv(override=True)

# Create a ChatOpenAI model
model = ChatOpenAI(model="gpt-4o", verbose=True)

## 1. Structured Output with langchain

<img src="images/langchain_structured_output.png" width=800px />

In [6]:
# Define schema

from pydantic import BaseModel, Field
class ResponseFormatter(BaseModel):
    """Always use this tool to structure your response to the user."""
    user: str = Field(description="User name")
    email: str = Field(description="User email")



# Bind schema to model
model_with_structure = model.with_structured_output(ResponseFormatter)

# Invoke the model to produce structured output that matches the schema
structured_output = model_with_structure.invoke('Семен васильевич зарегистрирвал адрес semen@mail')
print(structured_output)

user='Семен Васильевич' email='semen@mail'


In [9]:
# use function calling

model_with_tools = model.bind_tools([ResponseFormatter])
structured_output = model_with_tools.invoke('Семен васильевич зарегистрирвал адрес semen@mail')
print(structured_output)

content='' additional_kwargs={'tool_calls': [{'id': 'call_cuIVSHW7af8HymeoNput9tl9', 'function': {'arguments': '{"user":"Семен Васильевич","email":"semen@mail"}', 'name': 'ResponseFormatter'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 71, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_831e067d82', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-8605c2cb-560c-41c3-98ca-089da85ee3ca-0' tool_calls=[{'name': 'ResponseFormatter', 'args': {'user': 'Семен Васильевич', 'email': 'semen@mail'}, 'id': 'call_cuIVSHW7af8HymeoNput9tl9', 'type': 'tool_call'}] usage_metadata={'input_tokens': 71, 'output_tokens': 24, 'total_tokens': 95, 'input_token_details': {'audio': 0, 'cache_read':

In [12]:
# use json_mode

model_json = ChatOpenAI(model="gpt-4o", model_kwargs={ "response_format": { "type": "json_object" } })
structured_output = model_json.invoke('Семен васильевич зарегистрирвал адрес semen@mail. Выдели имя и емейл. Ответь ввиде JSON')
print(structured_output.content)

{
    "имя": "Семен Васильевич",
    "email": "semen@mail"
}


## 2. Tools and Function calling

<img src="images/langchain_function_calling.png" width=900px />

# Лучшие практики

При разработке инструментов, которые будут использоваться моделями, учитывайте следующее:

- Инструменты с хорошо подобранными названиями, корректной документацией и правильной типизацией легче использовать моделям.
- Разрабатывайте простые и узкоспециализированные инструменты, так как их проще использовать корректно.
- Используйте чат-модели, которые поддерживают API для вызова инструментов, чтобы максимально эффективно использовать инструменты.

In [51]:
from langchain_core.tools import tool
from pprint import pp

@tool
def multiply(a: int, b: int) -> int:
   """Multiply two numbers."""
   return a * b

# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

pp(multiply.args_schema.schema())

multiply.invoke({'a':4, 'b':5})

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
{'description': 'Multiply two numbers.',
 'properties': {'a': {'title': 'A', 'type': 'integer'},
                'b': {'title': 'B', 'type': 'integer'}},
 'required': ['a', 'b'],
 'title': 'multiply',
 'type': 'object'}


/var/folders/ck/mnjcv5_14nq5jzl5wb8kd5gh0000gn/T/ipykernel_37005/1104359578.py:14: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  pp(multiply.args_schema.schema())


20

In [25]:
# Tools from chain

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [("human", "Классифицируй запрос на один из типов: нейтральный, негативный, позитивный. Запрос: {request}.")]
)

# Placeholder LLM

chain = prompt | model | StrOutputParser()

as_tool = chain.as_tool(
    name="Feedback classifier", description="Classify user responces."
)
as_tool.args_schema.model_json_schema()

as_tool.invoke({'request': 'Отвратительный товар. НИкогда не куплю.'})

'Негативный.'

In [52]:
model_with_tool = model.bind_tools([multiply])

In [58]:

from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

messages = [HumanMessage(content='Сколько будет 3 умножить на 6?')]
response = model_with_tool.invoke(messages)
messages.append(response)

In [59]:
response.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 6},
  'id': 'call_DtKCBtoi8QKjUX3LsEOYLJby',
  'type': 'tool_call'}]

In [60]:
for tool_call in response.tool_calls:
    selected_tool = { "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
print(messages)

[HumanMessage(content='Сколько будет 3 умножить на 6?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_DtKCBtoi8QKjUX3LsEOYLJby', 'function': {'arguments': '{"a":3,"b":6}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 56, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_7f6be3efb0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-586215a6-3196-42b7-9d14-2a6e68161c0e-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 6}, 'id': 'call_DtKCBtoi8QKjUX3LsEOYLJby', 'type': 'tool_call'}], usage_metadata={'input_tokens': 56, 'output_tokens': 17, 'total_tokens': 73, 'input_token_details'

In [61]:
model_with_tool.invoke(messages)

AIMessage(content='3 умножить на 6 будет 18.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 81, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_7f6be3efb0', 'finish_reason': 'stop', 'logprobs': None}, id='run-f4d6fdb6-7984-47cb-9f7c-ab43ab857da8-0', usage_metadata={'input_tokens': 81, 'output_tokens': 12, 'total_tokens': 93, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})